In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as f

In [41]:
sequence_length = 4   # my name is ajay
batch_size = 1
input_dim = 512
d_model = 512
x = torch.rand((batch_size, sequence_length, input_dim))  # (1, 4, 512)   # x is input just after positional encoding

In [42]:
x.size()

torch.Size([1, 4, 512])

In [43]:
qkc_layer = nn.Linear(input_dim, 3*d_model)   # | Q K V |

In [44]:
qkv = qkc_layer(x)  # (1, 4, 3*512)
qkv.shape

torch.Size([1, 4, 1536])

# selfattention

In [45]:
num_heads = 8
head_dim = d_model // num_heads
qkv = qkv.reshape(batch_size, sequence_length, num_heads, 3*head_dim)  # (1, 4, 8, 3*64)

In [46]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [47]:
qkv = qkv.permute(0,2,1,3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [48]:
q, k, v = qkv.chunk(3, dim=-1)  ## breaking from 192 which 3*64
q.shape, k.shape, v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

In [49]:
import math
d_k = q.size()[-1]

#k.transpose(-2,-1)  becox 1, 8, 4, 64 to  1, 8, 64, 4

scaled = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k)  # (1, 8, 4, 4)
scaled.shape

torch.Size([1, 8, 4, 4])

In [50]:
mask = torch.full(scaled.size(), float('-inf')) # 1, 8, 4, 4
mask = torch.triu(mask, diagonal=1)  # 1, 8, 4, 4
mask

tensor([[[[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0.,

In [51]:
mask[0][1]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [52]:
(scaled + mask)[0][0]

tensor([[-0.0592,    -inf,    -inf,    -inf],
        [ 0.0727,  0.0216,    -inf,    -inf],
        [-0.0189, -0.0329, -0.0064,    -inf],
        [ 0.0337,  0.0310,  0.0980,  0.0339]], grad_fn=<SelectBackward0>)

In [53]:
scaled += mask

In [54]:
scaled.shape

torch.Size([1, 8, 4, 4])

In [55]:
attention = f.softmax(scaled, dim=-1)  # (1, 8, 4, 4)

In [56]:
attention.shape

torch.Size([1, 8, 4, 4])

In [57]:
attention[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5128, 0.4872, 0.0000, 0.0000],
        [0.3335, 0.3288, 0.3377, 0.0000],
        [0.2461, 0.2454, 0.2624, 0.2461]], grad_fn=<SelectBackward0>)

In [58]:
values = torch.matmul(attention, v)  # (1, 8, 4, 64)

In [59]:
values.shape

torch.Size([1, 8, 4, 64])

In [60]:
def scaled_dot_product(q,k,v, mask = None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    
    attention = f.softmax(scaled, dim=-1)  # apply only on last dimention
    values = torch.matmul(attention, v)

    return attention, values

attention, values = scaled_dot_product(q,k,v)
print(attention.shape, values.shape)

torch.Size([1, 8, 4, 4]) torch.Size([1, 8, 4, 64])
